In [12]:
# from google.colab import drive

# drive.mount("/content/gdrive")
# %cd /content/gdrive/MyDrive/AYR-ml_project_files/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/AYR-ml_project_files


In [13]:
# import zipfile
# import os

# def extract_zip(input_zip, output_dir):
#     with zipfile.ZipFile(input_zip, 'r') as zip_ref:
#         zip_ref.extractall(output_dir)

# # Usage example
# zip_path = 'data.zip'
# extract_to = ''
# extract_zip(zip_path, extract_to)


In [14]:
import os
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from augmentation import augs
from tqdm import tqdm

# Define transformations for the input data
transform = augs()

# Load your dataset
train_dataset = datasets.ImageFolder('data/train', transform=transform)
val_dataset = datasets.ImageFolder('data/validation', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=8)

# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Modify the classifier
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # Assuming 10 classes

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 50
best_accuracy = 0
best_epoch = 0
model_dir = 'model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Training and validation loop
for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Epoch {epoch+1} Validation Accuracy: {accuracy}%')

    # Checkpoint the best model
    if accuracy > best_accuracy and epoch >= 20:  # Save best model in the last 20 epochs
        best_accuracy = accuracy
        best_epoch = epoch
        model_path = os.path.join(model_dir, 'Resnet_best.pth')
        torch.save(model.state_dict(), model_path)
        print(f"Best model saved with accuracy: {best_accuracy}% at epoch {epoch+1}")

print(f"Training completed. Best model was at epoch {best_epoch+1} with an accuracy of {best_accuracy}%.")


Training Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1, Loss: 2.017865644880088


Training Epochs:   2%|▏         | 1/50 [00:15<12:42, 15.57s/it]

Epoch 1 Validation Accuracy: 54.60992907801418%
Epoch 2, Loss: 0.986552965928273


Training Epochs:   4%|▍         | 2/50 [00:30<12:15, 15.33s/it]

...

Epoch 48 Validation Accuracy: 94.8581560283688%
Epoch 49, Loss: 0.1550822476716151


Training Epochs:  98%|█████████▊| 49/50 [12:34<00:15, 15.48s/it]

Epoch 49 Validation Accuracy: 92.90780141843972%
Epoch 50, Loss: 0.13130699377787491
Epoch 50 Validation Accuracy: 97.16312056737588%


Training Epochs: 100%|██████████| 50/50 [12:50<00:00, 15.41s/it]

Best model saved with accuracy: 97.16312056737588% at epoch 50
Training completed. Best model was at epoch 50 with an accuracy of 97.16312056737588%.


In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
from augmentation import augs  # Ensure this matches your file structure
import os
from tqdm import tqdm

# Define transformations
transform = augs()

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the pre-trained ResNet-50 model
model_gradclip = models.resnet50(pretrained=True)

# Modify the classifier to match the number of classes in your dataset (10 classes)
num_ftrs = model_gradclip.fc.in_features
model_gradclip.fc = torch.nn.Linear(num_ftrs, 10)

# Load the saved model weights
model_path = 'model/Resnet_best.pth'
if os.path.exists(model_path):
    model_gradclip.load_state_dict(torch.load(model_path, map_location=device))
else:
    print("Model file not found.")
model_gradclip.to(device)
model_gradclip.eval()

# Prepare the dataset and DataLoader
test_dataset = datasets.ImageFolder(root='sample_eval_data', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Prepare to track class-wise accuracy
class_correct = list(0. for i in range(len(test_dataset.classes)))
class_total = list(0. for i in range(len(test_dataset.classes)))

# Evaluate the model
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model_gradclip(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Print class-wise accuracy
for i in range(len(test_dataset.classes)):
    if class_total[i] > 0:
        class_accuracy = 100 * class_correct[i] / class_total[i]
        print(f'Accuracy of {test_dataset.classes[i]}: {class_accuracy:.2f}%')
    else:
        print(f'Accuracy of {test_dataset.classes[i]}: N/A - No samples')

# Print overall accuracy|
total_correct = sum(class_correct)
total_samples = sum(class_total)
overall_accuracy = 100 * total_correct / total_samples if total_samples > 0 else 0
print(f'Overall accuracy of the model on test data: {overall_accuracy:.2f}%')


Using device: cpu


100%|██████████| 2/2 [00:03<00:00,  1.94s/it]

Accuracy of AguafinaScript: 100.00%
Accuracy of AlexBrush: 80.00%
Accuracy of Allura: 80.00%
Accuracy of Canterbury: 100.00%
Accuracy of GreatVibes: 60.00%
Accuracy of Holligate Signature: 100.00%
Accuracy of I Love Glitter: 100.00%
Accuracy of James Fajardo: 100.00%
Accuracy of OpenSans: 80.00%
Accuracy of alsscrp: 60.00%
Overall accuracy of the model on test data: 86.00%


In [18]:
# from google.colab import runtime
# runtime.unassign()